![Assimila logo](../../helpers/logosmall.png "Assimila")
# Assimila Peatlands COP26 Demonstrator

If the drop down menus do not appear restart the notebook by selecting "Kernel" then "Restart" above.

In [1]:
import sys
sys.path.append("../../helpers")
import os
from data import Data
from maptools import MapTools
from widgets import Widgets
import matplotlib.pyplot as plt
import matplotlib
%matplotlib notebook
# matplotlib.use("nbagg")
from IPython.display import clear_output, FileLink
from IPython.core.display import HTML
from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles, 
                        TileLayer, ImageOverlay, Polyline, Polygon, 
                        Rectangle,  GeoJSON, WidgetControl,  DrawControl, 
                        LayerGroup, FullScreenControl, interactive, ScaleControl)
import ipywidgets as widgets
import datetime
import csv
import subprocess

display(HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
"""))

display(HTML("""
<style>
.container { 
    width:70% !important; 
}
</style>
"""))

display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))


def get_product_name(product):
    """
    Return the DataCube standard name for a product after selection from 
    the dropdown menu.
    
    :param product: product name as given in the menu
    
    :return key: product ID
    """
    for key, value in w.projection_list.items():
        if value == product:
            return key

def on_button_clicked(b):
    """"
    Calls the appropriate method in the Data class depending on which operation has 
    been selected. Executed once the 'Get Data' button has been pressed.
    
    :return:
    """
    global m1
    d = Data(out, keyfile = None)
    button.disabled = True
    with out:
        clear_output()
        
    # Reproject coords to WSG84
    if east.value == 0 and west.value == 0 and south.value == 0 and north.value == 0:
        with out2:
            raise ValueError("You must select a point or area location on the map")
            button.disabled = False
            return

    east_, north_ = d.reproject_coords(east.value, north.value, projection.value)
    west_, south_ = d.reproject_coords(west.value, south.value, projection.value)

        
    Widgets.hide_save_options(save_map, save_format, save_data)
    
    
        
    if operation.value == 'Average of one sub-product':
        m1 = d.average_subproduct(get_product_name(product1.value), 
                                  subproduct1.value,
                                  frequency.value,
                                  average.value,
                                  north_,
                                  east_, 
                                  south_, 
                                  west_,
                                  date1.value, 
                                  date2.value,
                                  projection.value)

        if north.value == south.value and east.value == west.value:
            save_format.options = ['.csv']
        else:
            save_format.options = ['.nc', '.shp', '.png']
            
            # Add color image as a data layer to map
            # m.data_overlay(m1[0])
                
                
    if operation.value == 'Subtraction of one sub-product':
        m1 = d.color_map_subtraction(get_product_name(product1.value), 
                                      subproduct1.value, 
                                      north_,
                                      east_, 
                                      south_, 
                                      west_,
                                      date1.value, 
                                      date2.value)
        
        if north.value == south.value and east.value == west.value:
            save_format.options = ['.csv']
        else:
            save_format.options = ['.nc', '.shp', '.png']
            
            # Add color image as a data layer to map
            
    if operation.value == 'Trend analysis for one sub-product':
        m1 = d.trend_analysis(get_product_name(product1.value), 
                              subproduct1.value, 
                              north_, 
                              east_, 
                              south_, 
                              west_, 
                              date1.value, 
                              date2.value, 
                              date3.value, 
                              date4.value,
                              trends.value)
        
        if north.value == south.value and east.value == west.value:
            save_format.options = ['.csv', '.nc', '.png']
        else:
            save_format.options = ['.nc', '.shp', '.png']
            
            
    if operation.value == 'Identifying change':
        dates = []
        for item in date_carousel.children:
            dates.append(item.value)
        
        m1 = d.color_map_identifying_change(get_product_name(product1.value), 
                                            subproduct1.value,
                                            get_product_name(product2.value),
                                            subproduct2.value,
                                            north_, 
                                            east_,
                                            south_, 
                                            west_, 
                                            dates)
        
        if north.value == south.value and east.value == west.value:
            save_format.options = ['.csv']
        else:
            save_format.options = ['.nc', '.shp', '.png']
        
    button.disabled = False
    
    if m1 is not None:   
        Widgets.show_save_options(save_map, save_format, save_data)
    
def csv_writer(path, names, values):
    """
    Write a csv file for an arbitrary number of names and values.
    Each name/value pair forms a row in the csv file.
    
    :param names:  list of names of values
    :param values: list of values
    """
    with open(path, mode='w') as f:
        writer = csv.writer(f, delimiter=',')
        for n, v in zip(names, values) :
            writer.writerow([n, v])
    
def on_loc_button_clicked(b):
    """"
    Adds a marker/rectangular bounding box to the map when manual coordinates
    are entered and the 'get location' button is clicked.
    
    :return:
    """

    if north.value and not south.value and east.value and not west.value:
        m.add_map_point(east.value, north.value)
        m.map.center = (north.value, east.value)
        m.map.zoom = 7
        
    elif not north.value and south.value and not east.value and west.value:
        m.add_map_point(east.value, north.value)
        m.map.center = (south.value, west.value)
        m.map.zoom = 7
        
    elif north.value and east.value and south.value and west.value:
        if north.value == south.value and east.value == west.value:
            m.add_map_point(east.value, north.value)
            m.map.center = (south.value, west.value)
            m.map.zoom = 7
        else:
            m.add_map_rect(north.value, east.value, south.value, west.value)
            m.map.center = (float((north.value+south.value)/2), float((east.value+west.value)/2))
            m.map.zoom = 7
            
def on_save_map_clicked(b):
    """
    Saves a static HTML of the map and layers when the 'save map' button is clicked.
    
    :return:
    """
    
    path = '../../helpers/files/maps/map.html'
    m.map.save(path)
    display(FileLink(path))
        
    
def on_save_data_clicked(b):
    """
    Saves a csv, netCDF or shp file when 'save data' button is clicked.
    
    :return:
    """
    filename = f"{product1.value}_{subproduct1.value}_{north.value}" \
               f"_{east.value}_{date1.value}_{date2.value}"

    if operation.value == 'Average of one sub-product':
        if north.value == south.value and east.value == west.value:
            xr = m1
            path = os.path.join('../../helpers/files/csv/', filename + '.csv')
            names = ['Product', 'Subproduct', 'Lat', 'Lon',
                     'Date 1', 'Date 2', 'Average']
            values = [product1.value, subproduct1.value, north.value,
                     east.value, date1.value, date2.value, xr.data]
            csv_writer(path, names, values) # W
            display(FileLink(path))
            
        else:      
            xr, fig = m1
                
            if save_format.value == '.png':
                path = os.path.join('../../helpers/files/figs/', filename + '.png')
                fig.savefig(path) # W
                display(FileLink(path))
                
            elif save_format.value == '.nc':
                path = os.path.join('../../helpers/files/netCDF/', filename + '.nc')
                xr.to_netcdf(path) # W
                display(FileLink(path))
            else:
                pass
#                 d.geojson_to_shape(input_f, output_f)
    
    if operation.value == 'Subtraction of one sub-product':
        if north.value == south.value and east.value == west.value:
            xr = m1
            path = os.path.join('../../helpers/files/csv/', filename + '.csv')
            names = ['Product', 'Subproduct', 'Lat', 'Lon',
                     'Date 1', 'Date 2', 'Change']
            values = [product1.value, subproduct1.value, north.value,
                     east.value, date1.value, date2.value, xr.data]
            csv_writer(path, names, values) # W
            display(FileLink(path))
            
        else:
            xr, fig = m1
            if save_format.value == '.png':
                path = os.path.join('../../helpers/files/figs/', filename + '.png')
                fig.savefig(path) # W
                display(FileLink(path))

            elif save_format.value == '.nc':
                path = os.path.join('../../helpers/files/netCDF/', filename + '.nc')
                xr.to_netcdf(path) # W
                display(FileLink(path))
             
            else:
                # to shapefile
                pass
            
    if operation.value == 'Trend analysis for one sub-product':
        filename2 = f"{product1.value}_{subproduct1.value}_{north.value}" \
                    f"_{east.value}_{date3.value}_{date4.value}"
        
        xr1, xr2, fig = m1
        if save_format.value == '.png':
            path = os.path.join('../../helpers/files/figs/', filename + '.png')
            fig.savefig(path) # W
            display(FileLink(path))
            
        elif save_format.value == '.nc':
            path1 = os.path.join('../../helpers/files/netCDF/', filename + '.nc')
            path2 = os.path.join('../../helpers/files/netCDF/', filename2 + '.nc')
            
            xr1.to_netcdf(path1) # W     # NW for timeseries
            xr1.to_netcdf(path2) # W
            
            display(FileLink(path1))
            display(FileLink(path2))
        
        elif save_format.value == '.csv':
            path1 = os.path.join('../../helpers/files/csv/', filename)
            path2 = os.path.join('../../helpers/files/csv/', filename2)

            xr1.to_dataframe().to_csv(path1) # W
            xr2.to_dataframe().to_csv(path2) # W
            
            display(FileLink(path1))
            display(FileLink(path2))
        
        else:
            # to .shp
            pass
                
    if operation.value == 'Identifying change':
        filename1 = f"{product1.value}_{subproduct1.value}_{north.value}_{east.value}"
        filename2 = f"{product2.value}_{subproduct2.value}_{north.value}_{east.value}"
        
        if north.value == south.value and east.value == west.value:
            xr_arr1, xr_arr2 = m1
            
            # Define 2 paths for files based on the data they are going to contain
            path1 = os.path.join('../../helpers/files/csv/', filename1 + '_' + str(date_carousel.children[0].value) + 
                                 '_' + str(date_carousel.children[-1].value) + '.csv')
            
            path2 = os.path.join('../../helpers/files/csv/', filename2 + '_' + str(date_carousel.children[0].value) + 
                                 '_' + str(date_carousel.children[-1].value) + '.csv')
            
            # Write first csv with 1st product data
            names = ['Product 1', 'Subproduct 1', 'Lat', 'Lon'] + [d.value for d in date_carousel.children]
            values = [product1.value, subproduct1.value, north.value, east.value] + [xr_arr1[i][subproduct1.value][0].data for i in range(len(xr_arr1))]
            csv_writer(path1, names, values) # W
            
            # Write second csv with 2nd product data 
            names = ['Product 2', 'Subproduct 2', 'Lat', 'Lon'] + [d.value for d in date_carousel.children]
            values = [product2.value, subproduct2.value, north.value, east.value] + [xr_arr2[i][subproduct2.value][0].data for i in range(len(xr_arr2))]
            csv_writer(path2, names, values) # W
            
        else:
            xr_arr1, fig1, x_arr2, fig2 = m1
            
            if save_format.value == '.png':
                path1 = os.path.join('../../helpers/files/figs/', filename1 + '_' + str(date_carousel.children[0].value) + 
                                     '_' + str(date_carousel.children[-1].value) + '.png')
                
                path2 = os.path.join('../../helpers/files/figs/', filename2 + '_' + str(date_carousel.children[0].value) + 
                                     '_' + str(date_carousel.children[-1].value) + '.png')
                
                fig1.savefig(path1) # W
                display(FileLink(path1))
                
                fig2.savefig(path2) # W
                display(FileLink(path2))
                    
            elif save_format.value == '.nc': 
                
                i=1
                for count, xr in enumerate(xr_arr1):
                    path = os.path.join('../../helpers/files/netCDF/', filename + str(date_carousel.children[i].value) + '.nc')
                    xr[subproduct1.value][0].to_netcdf(path) # NW
                    display(FileLink(path))
                    i+=1

                for xr in xr_arr2:
                    path = os.path.join('../../helpers/files/netCDF/', filename + str(date_carousel.children[i].value) + '.nc')
                    xr[subproduct1.value].to_netcdf(path) # NW
                    display(FileLink(path))
                    i+=1
            else:
                # to shapefile
                pass

def update_north(x):
    """"
    Updates the 'North' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (north, 'value'))

def update_east(x):
    """"
    Updates the 'East' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (east, 'value'))

def update_south(x):
    """"
    Updates the 'South' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (south, 'value'))

def update_west(x):
    """"
    Updates the 'West' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (west, 'value'))    
    
def handle_draw(self, action, geo_json):
    """
    Handles drawing of layers on the map by extracting coordinates
    and updating the relevant coordinate display box.
    
    :param action: 
    :param geo_json: 
    
    :return:
    """
    global trend_criteria1
    
    w = Widgets()
    d = Data(out)
    
    # Remove manual layers before drawing 
    if len(m.layers_list) != 0:
        m.map.remove_layer(m.layers_list.pop(0))
    
    s = geo_json.get('geometry','type')
    t = s.get('type')
    
    if t == 'Point':
        _north, _east, _south, _west = m.get_coords_point(geo_json)
        north, east, south, west = d.check_coords(_north, _east, _south, _west, projection.value)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)
        trend_criteria1 = True

    if t == 'Polygon':
        _north, _east, _south, _west = m.get_coords_polygon(geo_json)
        north, east, south, west = d.check_coords(_north, _east, _south, _west, projection.value)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west) 
        trend_criteria1 = False
        
    if trend_criteria1 and trend_criteria2:
        trends.disabled = False
        trends.layout.visibility = 'visible'
    else:
        trends.disabled = True
        trends.layout.visibility = 'hidden'
        
def update(*args):
    """
    Event function called when the user interacts with the product
    widgets allowing displays to be updated accordingly.
    
    :return:
    """
    if product1.value == ' ':
        subproduct1.layout.visibility = 'hidden'

    elif operation.value != 'Trend analysis for one sub-product':
        subproduct1.layout.visibility = 'visible'
        subproduct1.options = w.get_subproduct_list(product1.value, peat=True)
    
    else:
        subproduct1.layout.visibility = 'visible'
        
    if product2.value == ' ' or operation.value != 'Identifying change':
        subproduct2.layout.visibility = 'hidden'
    
    else:
        subproduct2.layout.visibility = 'visible'
        subproduct2.options = w.get_subproduct_list(product2.value, peat=True)
    
    if operation.value == 'Subtraction of one sub-product':
        product2.layout.visibility = 'hidden'
        subproduct2.layout.visibility = 'hidden'
    
    if operation.value == 'Trend analysis for one sub-product':
        subproduct1.options = w.get_subproduct_trend_analysis(product1.value)
    
    
def update_proj(*args):
    """
    Event function called when the user interacts with the projection
    widget, allowing coordinate displays to be converted.
    
    :return:
    """
    new_proj = args[0]['new']
    old_proj = args[0]['old']

    d = Data(out)
    
    if new_proj == 'Lat/Lon' and old_proj == 'National Grid':
        x1, y1 = d.coord_transform(x=east.value, y=north.value, conv='bng_to_latlon')
        x2, y2 = d.coord_transform(x=west.value, y=south.value, conv='bng_to_latlon')
        
        update_north(y1)
        update_east(x1)
        update_south(y2)
        update_west(x2)
        
        north.description = 'North (lat):'
        east.description = 'East (lon):'
        south.description = 'South (lat):'
        west.description = 'West (lon):'

    if new_proj == 'National Grid' and old_proj == 'Lat/Lon':
        x1, y1 = d.coord_transform(x=east.value, y=north.value, conv='latlon_to_bng')
        x2, y2 = d.coord_transform(x=west.value, y=south.value, conv='latlon_to_bng')
        
        update_north(y1)
        update_east(x1)
        update_south(y2)
        update_west(x2)
        
        north.description = 'North (nort):'
        east.description = 'East (east):'
        south.description = 'South (nort):'
        west.description = 'West (east):'
        
    if new_proj == 'Sinusoidal' and old_proj == 'National Grid':
        x1, y1 = d.coord_transform(x=east.value, y=north.value, conv='bng_to_sinu')
        x2, y2 = d.coord_transform(x=west.value, y=south.value, conv='bng_to_sinu')
            
        update_north(y1)
        update_east(x1)
        update_south(y2)
        update_west(x2)
        
        north.description = 'North:'
        east.description = 'East:'
        south.description = 'South:'
        west.description = 'West:'
        
        
    if new_proj == 'National Grid' and old_proj == 'Sinusoidal':
        x1, y1 = d.coord_transform(x=east.value, y=north.value, conv='sinu_to_bng')
        x2, y2 = d.coord_transform(x=west.value, y=south.value, conv='sinu_to_bng')
        
        update_north(y1)
        update_east(x1)
        update_south(y2)
        update_west(x2)
        
        north.description = 'North (nort):'
        east.description = 'East (east):'
        south.description = 'South (nort):'
        west.description = 'West (east):'
        

    if new_proj == 'Sinusoidal' and old_proj == 'Lat/Lon':
        x1, y1 = d.coord_transform(x=east.value, y=north.value, conv='latlon_to_sinu')
        x2, y2 = d.coord_transform(x=west.value, y=south.value, conv='latlon_to_sinu')       

        update_north(y1)
        update_east(x1)
        update_south(y2)
        update_west(x2)
        
        north.description = 'North:'
        east.description = 'East:'
        south.description = 'South:'
        west.description = 'West:'

    if new_proj == 'Lat/Lon' and old_proj == 'Sinusoidal':
        x1, y1 = d.coord_transform(x=east.value, y=north.value, conv='sinu_to_latlon')
        x2, y2 = d.coord_transform(x=west.value, y=south.value, conv='sinu_to_latlon')
        
        update_north(y1)
        update_east(x1)
        update_south(y2)
        update_west(x2)
        
        north.description = 'North (lat):'
        east.description = 'East (lon):'
        south.description = 'South (lat):'
        west.description = 'West (lon):'
        
def update_file(*args):
    """
    Event function called when the user interacts with the file 
    upload widget. Files are written to the server, converted to 
    .geojson format and added to the map using Data and Maptools 
    methods.
    
    :return:
    """
    change = args[0]
    
    # Get shapefile name and pass into func
    fnames = list(change['owner'].value)
    
    # Write file locally 
    for count, fname in enumerate(fnames):
        loc = os.path.join('../../helpers/files/shp/', fname)
        with open(loc, 'wb') as outf:
            content = change['owner'].data[count]
            outf.write(content)
    
    # Skip conversion if already .geojson
    if '.geojson' in fname:
        m.add_geojson(loc)
    
    else:
        # convert .shp to .geojson
        x = os.path.join('../../helpers/files/shp', fname)
        y = os.path.join('../../helpers/files/shp/', 'POLYGON.geojson')
        
        Data.shape_to_geojson(x, y)
        north, east, south, west = m.add_geojson(os.path.join('../../helpers/files/shp/', 'POLYGON.geojson'))

        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)
    
    file_upload.value.clear()
    file_upload._counter = 0
    subprocess.call('rm ../../helpers/files/shp/*', shell = True)
    
def update_operation(*args): 
    """
    Event function called when the user interacts with the operation
    widget. Other widgets are disabled/enabled & hidden/shown 
    based on the user requirements of each analysis operation. 
    
    :return:
    """
    global trend_criteria2
    trend_criteria2 = False
    
    change = args[0]
    operation = change['new']
    if operation == 'Average of one sub-product':
        product1.options = default_options
        product2.disabled = True
        product2.layout.visibility = 'hidden'
        subproduct2.disabled = True
        subproduct2.layout.visibility = 'hidden'
        
        date1.disabled = False
        date1.layout.visibility = 'visible'
        date2.disabled = False
        date2.layout.visibility = 'visible'
        date3.disabled = True
        date3.layout.visibility = 'hidden'
        date4.disabled = True
        date4.layout.visibility = 'hidden'
        
        average.disabled = False
        average.layout.visibility = 'visible'
        trends.disabled = True
        frequency.disabled = False
        frequency.layout.visibility = 'visible'
        frequency.options = [' ', 'days', 'months', 'years']
        date_carousel.layout.visibility = 'hidden'
        
    elif operation == 'Subtraction of one sub-product':
        product1.options = default_options
        product2.disabled = True
        product2.layout.visibility = 'hidden'
        subproduct2.disabled = True
        subproduct2.layout.visibility = 'hidden'
        
        date1.disabled = False
        date1.layout.visibility = 'visible'
        date2.disabled = False
        date2.layout.visibility = 'visible'
        date3.disabled = True
        date3.layout.visibility = 'hidden'
        date4.disabled = True
        date4.layout.visibility = 'hidden'

        average.disabled = True
        average.layout.visibility = 'hidden'
        trends.disabled = True
        frequency.disabled = True
        frequency.layout.visibility = 'hidden'
        date_carousel.layout.visibility = 'hidden'
    
    if operation == 'Trend analysis for one sub-product':
        product1.options = [' ', 'Vegetation indices', 'Albedo', 'Temperature']
        product2.disabled = True
        product2.layout.visibility = 'hidden'
        subproduct2.disabled = True
        subproduct2.layout.visibility = 'hidden'
        
        date1.disabled = False
        date1.layout.visibility = 'visible'
        date2.disabled = False
        date2.layout.visibility = 'visible'
        date3.disabled = False
        date3.layout.visibility = 'visible'
        date4.disabled = False
        date4.layout.visibility = 'visible'
        
        average.disabled = True
        average.layout.visibility = 'hidden'
        trend_criteria2 = True
        frequency.disabled = True
        frequency.layout.visibility = 'hidden'
        date_carousel.layout.visibility = 'hidden'

    if operation == 'Identifying change':
        product1.options = default_options
        product2.disabled = False
        product2.value = ' '
        product2.layout.visibility = 'visible'
        subproduct2.layout.visibility = 'hidden'
        subproduct2.disabled = False
        
        date1.disabled = True
        date1.layout.visibility = 'hidden'
        date2.disabled = True
        date2.layout.visibility = 'hidden'
        date3.disabled = True
        date3.layout.visibility = 'hidden'
        date4.disabled = True
        date4.layout.visibility = 'hidden'

        average.disabled = True
        average.layout.visibility = 'hidden'
        trends.disabled = True
        trends.layout.visibility = 'hidden'
        frequency.disabled = False
        frequency.layout.visibility = 'visible'
        frequency.options = [' ', 2, 3, 4, 5]
        date_carousel.layout.visibility = 'visible'
        
    if trend_criteria1 and trend_criteria2:
        trends.disabled = False
        trends.layout.visibility = 'visible'
    else:
        trends.disabled = True
        trends.layout.visibility = 'hidden'
        
def update_frequency(*args):
    """
    Event function called when the user interacts with the frequency
    widget. A scrollable date carousel of dates is populated depending
    on how many dates the user requires.
    
    :return:
    """
    
    if operation.value != 'Identifying change':
        return
    n = args[0]['new']
    dates = []
    for i in range(n):
        dates.append(w.get_date(value=datetime.date(2018,1,1), description=f'Date {i+1}:', layout='Default'))
    date_carousel.children = dates

trend_criteria1, trend_criteria2 = False, False      

# Initialize map and add DrawControl() 
m = MapTools(center=(55, -3.7), zoom=7, width='480px', height='775px', os_api=True)
m.map.add_control(m.dc)
m.map.add_control(ScaleControl(position='bottomleft'))
m.prepare_map()

# Initialize widgets 
w = Widgets()

operation = w.operation()

product1 = w.product('Satellite Data 1:', 'product1', peat=True)
subproduct1 = w.subproduct('Measurement 1:', 'subproduct1')

product2 = w.product('Satellite Data 2:', 'product2', peat=True)
subproduct2 = w.subproduct('Measurement 2:', 'subproduct2')

projection = w.projection()
file_upload = w.upload_file()

average = w.average()
trends = w.trends()
frequency = w.frequency()
date_carousel = w.date_carousel()

save_format = w.save_format()

north = w.get_point(None, 'North (lat): ')
east = w.get_point(None, 'East (lon): ')
south = w.get_point(None, 'South (lat): ')
west = w.get_point(None, 'West (lon): ')

date1 = w.get_date(datetime.date(2018,1,1), 'Start 1: ', 'date12')
date2 = w.get_date(datetime.date(2018,1,17), 'End 1: ', 'date12')
date3 = w.get_date(datetime.date(2018,1,1), 'Start 2: ', 'date34')
date4 = w.get_date(datetime.date(2018,1,17), 'End 2: ', 'date34')

button = w.set_up_button(method=on_button_clicked, description="Get data")
button_loc = w.set_up_button(method=on_loc_button_clicked, description="Get location")
save_map = w.set_up_button(method=on_save_map_clicked, description="Save map", layout='save')
save_data = w.set_up_button(method=on_save_data_clicked, description="Save data", layout='save')

Widgets.hide_save_options(save_map, save_format, save_data)

Widgets.display_widget_comparison_reduced(operation, product1, subproduct1, product2, subproduct2, 
                                    projection, date_carousel, north, east,south, west, button_loc, date1, 
                                    date2, date3, date4, file_upload, button, m.map, average, trends, 
                                    frequency, save_map, save_format, save_data)

default_options = list(w.projection_list.values())

# Link events to update functions
label = widgets.Label()
m.mouse_interaction(label)
        
update()

product1.observe(update)
product2.observe(update)
subproduct1.observe(update)

projection.observe(update_proj, names='value')
file_upload.observe(update_file, names='data')
operation.observe(update_operation, names='value')
frequency.observe(update_frequency, names='value')

out = w.display_output()

x = m.dc.on_draw(handle_draw) 

#############
# Plot Area #
#############

output_layout = widgets.Layout(width='100%', height='950px', border='5px solid grey')
out2 = w.display_output()
out2.layout = output_layout

with out2:
    display(widgets.HTML(value="</b><big><u> Plot Area </u></big></b>"))

Label(value='')

Output()

Output()

In [35]:
ds = m1[0]
print(ds)
conv = "sinu_to_latlon"
d = Data(out)

ds["latitude"] = [d.coord_transform(0, i, conv)[1] for i in m1[0]["latitude"].data]
ds["longitude"] = [d.coord_transform(i, 0, conv)[0] for i in m1[0]["longitude"].data]

print(ds)

<xarray.DataArray 'LST_Day_1km' (latitude: 70, longitude: 82)>
array([[      nan,       nan,       nan, ..., 273.66   , 273.59332, 269.16   ],
       [      nan,       nan,       nan, ..., 274.63   , 270.55667, 270.63   ],
       [      nan,       nan,       nan, ..., 274.65   , 274.675  , 269.60858],
       ...,
       [271.58667, 271.80667, 271.86   , ...,       nan,       nan,       nan],
       [272.00333, 272.10666, 271.45667, ..., 275.675  ,       nan,       nan],
       [273.64398, 274.58002, 272.58798, ..., 275.9571 ,       nan,       nan]],
      dtype=float32)
Coordinates:
  * latitude   (latitude) float64 6.159e+06 6.158e+06 ... 6.096e+06 6.095e+06
  * longitude  (longitude) float64 -3.187e+05 -3.178e+05 ... -2.436e+05
<xarray.DataArray 'LST_Day_1km' (latitude: 70, longitude: 82)>
array([[      nan,       nan,       nan, ..., 273.66   , 273.59332, 269.16   ],
       [      nan,       nan,       nan, ..., 274.63   , 270.55667, 270.63   ],
       [      nan,       nan,       n